In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import date

In [2]:
df_train=pd.read_csv('train.csv')

In [3]:
df_train=df_train.dropna()

In [4]:
df_train.shape

(225493, 41)

In [5]:
df_train.shape

(225493, 41)

In [6]:
df_train.select_dtypes('object').columns

Index(['Date.of.Birth', 'Employment.Type', 'DisbursalDate',
       'PERFORM_CNS.SCORE.DESCRIPTION', 'AVERAGE.ACCT.AGE',
       'CREDIT.HISTORY.LENGTH'],
      dtype='object')

In [7]:
def cns_desc(x):
    if x<300:
        return 0
    elif (x>=300) and (x<=350):
        return 1
    elif (x>350) and (x<=570):
        return 2
    elif (x>570) and (x<=630):
        return 3
    elif (x>630) and (x<=705):
        return 4
    else:
        return 5

In [8]:
def fix_date(x):

    if x.year >= 2049:
        year = x.year - 100
    else:
        year = x.year
    return datetime.date(year,x.month,x.day)
df_train["Date.of.Birth"]=pd.to_datetime(df_train["Date.of.Birth"])
df_train["Date.of.Birth"] = df_train["Date.of.Birth"].apply(fix_date)



In [9]:
df_train["Date.of.Birth"]=pd.to_datetime(df_train["Date.of.Birth"])

df_train["DisbursalDate"]=pd.to_datetime(df_train["DisbursalDate"])
df_train["DisbursalDate_Month"]=df_train["DisbursalDate"].dt.month

df_train["Age_at_disbursement"]=((df_train['DisbursalDate']-df_train["Date.of.Birth"])/365).apply(lambda x:x.days)

df_train["Employment.Type"]=df_train["Employment.Type"].map({'Salaried':0,"Self employed":1,np.nan:2})

df_train["PERFORM_CNS.SCORE.DESCRIPTION"]=df_train["PERFORM_CNS.SCORE"].apply(cns_desc)

df_train["AVERAGE.ACCT.AGE"]=df_train["AVERAGE.ACCT.AGE"].apply(lambda x:(int((x.split("y"))[0])*12)+(int((x.split())[1].split("m")[0])))

df_train["CREDIT.HISTORY.LENGTH"]=df_train["CREDIT.HISTORY.LENGTH"].apply(lambda x:(int((x.split("y"))[0])*12)+(int((x.split())[1].split("m")[0])))

In [10]:
df_train['NO.OF.ACCTS']=df_train['PRI.NO.OF.ACCTS']+df_train['SEC.NO.OF.ACCTS']
df_train['ACTIVE.ACCTS']=df_train['PRI.ACTIVE.ACCTS']+df_train['SEC.ACTIVE.ACCTS']
df_train['OVERDUE.ACCTS']=df_train['PRI.OVERDUE.ACCTS']+df_train['SEC.OVERDUE.ACCTS']
df_train['CURRENT.BALANCE']=df_train['PRI.CURRENT.BALANCE']+df_train['SEC.CURRENT.BALANCE']
df_train['SANCTIONED.AMOUNT']=df_train['PRI.SANCTIONED.AMOUNT']+df_train['SEC.SANCTIONED.AMOUNT']
df_train['DISBURSED.AMOUNT']=df_train['PRI.DISBURSED.AMOUNT']+df_train['SEC.DISBURSED.AMOUNT']
df_train['INSTAL.AMT']=df_train['PRIMARY.INSTAL.AMT']+df_train['SEC.INSTAL.AMT']

df_train=df_train.drop(['PRI.NO.OF.ACCTS','SEC.NO.OF.ACCTS','PRI.ACTIVE.ACCTS','SEC.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS','SEC.OVERDUE.ACCTS','PRI.CURRENT.BALANCE','SEC.CURRENT.BALANCE','PRI.SANCTIONED.AMOUNT','SEC.SANCTIONED.AMOUNT','PRI.DISBURSED.AMOUNT','SEC.DISBURSED.AMOUNT','PRIMARY.INSTAL.AMT','SEC.INSTAL.AMT'],axis=1)

In [11]:
df_train=df_train.drop(columns=['Date.of.Birth',"DisbursalDate"])

In [12]:
df_train[df_train['DISBURSED.AMOUNT']==df_train['DISBURSED.AMOUNT'].max()]

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Employment.Type,State_ID,...,loan_default,DisbursalDate_Month,Age_at_disbursement,NO.OF.ACCTS,ACTIVE.ACCTS,OVERDUE.ACCTS,CURRENT.BALANCE,SANCTIONED.AMOUNT,DISBURSED.AMOUNT,INSTAL.AMT
71593,598208,47773,85964,59.33,20,16700,86,6434,1,5,...,1,10,50,2,1,0,36041,1000000000,1000000000,0


In [13]:
df_train=df_train.drop([71593])

In [14]:
df_train.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Employment.Type,State_ID,...,loan_default,DisbursalDate_Month,Age_at_disbursement,NO.OF.ACCTS,ACTIVE.ACCTS,OVERDUE.ACCTS,CURRENT.BALANCE,SANCTIONED.AMOUNT,DISBURSED.AMOUNT,INSTAL.AMT
0,420825,50578,58400,89.55,67,22807,45,1441,0,6,...,0,3,34,0,0,0,0,0,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,1,6,...,1,9,33,1,1,1,27600,50200,50200,1991
2,417566,53278,61360,89.63,67,22807,45,1497,1,6,...,0,1,32,0,0,0,0,0,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,1,6,...,1,10,24,3,0,0,0,0,0,31
4,539055,52378,60300,88.39,67,22807,45,1495,1,6,...,1,9,41,0,0,0,0,0,0,0


In [15]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import KFold,cross_val_score

In [16]:
X=df_train.drop(['loan_default'],axis=1)
y=df_train['loan_default']

In [18]:
X=X.drop(['UniqueID'],axis=1)

### Model performance without Tuning

In [22]:
adc=AdaBoostClassifier()
kfold=KFold(n_splits=5, shuffle=True, random_state=1)
scores=cross_val_score(adc,X,y,cv=kfold,scoring='roc_auc',n_jobs=-1)

In [23]:
print(f'bias error',np.mean(scores))

bias error 0.6488453115562705


In [24]:
print(f'varaince error',np.std(scores,ddof=1))

varaince error 0.0023429983277675026


### Model performance with Tuning

In [ ]:
for i in np.arange(700,1000,100):
    print(i)
    adc=AdaBoostClassifier(n_estimators=i,random_state=1)
    kfold=KFold(n_splits=5, shuffle=True, random_state=1)
    scores=cross_val_score(adc,X,y,cv=kfold,scoring='roc_auc',n_jobs=-1)
    print(f'bias error',np.mean(scores))
    print(f'varaince error',np.std(scores,ddof=1))
    print(scores)

700


In [21]:
for i in [400]:
    print(i)
    adc=AdaBoostClassifier(n_estimators=i,random_state=1)
    kfold=KFold(n_splits=5, shuffle=True, random_state=1)
    scores=cross_val_score(adc,X,y,cv=kfold,scoring='roc_auc',n_jobs=-1)
    print(f'bias error',np.mean(scores))
    print(f'varaince error',np.std(scores,ddof=1))
    print(scores)

400
bias error 0.6580672508995671
varaince error 0.002111322804662625
[0.65514308 0.66105297 0.65851403 0.65794848 0.6576777 ]


In [ ]:
LD()